In [ ]:
import pandas as pd
from PIL import Image
import numpy as np
import glob
from matplotlib import pylab as plt
from multiprocessing import Pool
import time
import cv2
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
type_dict = {"Normal":0, "Fire":1, "Water":2, "Electric":3, "Grass":4, "Ice":5,
    "Fighting":6, "Poison":7, "Ground":8, "Flying":9, "Psychic":10, "Bug":11,
    "Rock":12, "Ghost":13, "Dragon":14, "Dark":15, "Steel":16,"Fairy":17 }
def read_types(path):
    """タイプデータを読み込んで返す.

    渡されたパスからデータを読み込み，タイプデータのみを抽出，整形し返す.

    Args:
        path (str) : データパス

    Returns:
         Numpy arrays. タイプデータリストを二つ

    """
    print(f'\rloading types data ... ', end='')
    df = pd.read_csv(path+'Pokemon.csv', sep=',')
    df.drop_duplicates(subset='Number', inplace=True)
    df.reset_index(inplace=True, drop=True)
    ind = df[df['Type2'].isnull()]['Type2'].index
    df.iloc[ind, 3] = df.iloc[ind, 2]

    df_1 = df["Type1"][:801]
    df_1 = df_1.map(type_dict)
    type1 = df_1.values

    df_2 = df["Type2"][:801]
    df_2 = df_2.map(type_dict)
    type2 = df_2.values

    print(f'\rloading types data ... done')
    return type1, type2

In [ ]:
type1,type2 = read_types('data/')
# print(type1)

In [ ]:
from sklearn import svm, metrics, preprocessing, model_selection #機械学習用のライブラリを利用
from mlxtend.plotting import plot_decision_regions #学習結果をプロットする外部ライブラリを利用
import codecs
import dataset
from sklearn.model_selection import train_test_split
with codecs.open('data/convert_data//No_Mega_Weight_new.csv', 'r', 'utf-8', 'ignore') as f:
    df_pokemon_all = pd.read_csv(f)
df_pokemon_all


In [ ]:
Type1 = df_pokemon_all.iloc[:,2]
#Type_change = np.array(list(weight))

#change_li = Type_change.reshape(-1,1)
Type2 = df_pokemon_all.iloc[:,3]

In [ ]:
import dataset_image
X = dataset_image.Data_set()
X1 = np.array(list(X))
X1 = X1.reshape(X1.shape[0], -1)
df = pd.DataFrame(X1)
df_change = pd.concat([df_pokemon_all, df],axis=1)
df_change
weight = df_change.iloc[:,13:27]
train_data = weight.values.tolist()
print(train_data)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import dataset_image

#print(X1)
train_X1 = train_data
train_y1 = Type1
(train_X1, test_X1 ,train_y1, test_y1) = train_test_split(train_X1, train_y1, test_size = 0.2, random_state = 666)
#print(train_X1)

In [ ]:
clf1 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf1 = clf1.fit(train_X1, train_y1)
pred1 = clf1.predict(test_X1)
print(f"acc: {clf1.score(test_X1, test_y1)}")


In [ ]:
for predx,test_y in zip(pred1,test_y1):
    
    print(f"pred : {predx}------",f"test : {test_y}")


In [ ]:
#説明変数XとタイプのみのY分離
import pandas as pd
poke = pd.read_csv('data/convert_data/No_Mega_Weight_new.csv')
poke_noType = poke.drop(['Number','Type1','Type2','Total','HP','Attack','Defense','SpecialAtk','SpecialDef','Speed','Generation','Legendary'],axis=1)
X = []
Y = []
Y_type2 = []
l = len(poke)
col = poke_noType.columns.values
coll = len(col)
index = 0
while index < l:
    column = 0
    Xdata = []
    Y_nt = []
    while column < coll: 
        Xdata.append(poke_noType.iat[index,column])
        column += 1

    Y_nt.append(poke.at[index,'Type1'])
    if type(poke.at[index,'Type2']) is not type(poke.at[index,'Type1']):
        Y_nt.append(poke.at[index,'Type1'])
    else:
        Y_nt.append(poke.at[index,'Type2'])
    X.append(Xdata)
    Y.append(Y_nt)
    index += 1

for i in range(l):
    Y_type2.append(Y[i][1])

    
print(Y_type2)


In [ ]:
train_X2 = train_data
train_y2 = Y_type2
(train_X2, test_X2 ,train_y2, test_y2) = train_test_split(train_X2, train_y2, test_size = 0.2, random_state = 666)
#print(train_X1)
clf2 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf2 = clf2.fit(train_X2, train_y2)
pred2 = clf2.predict(test_X2)
print(f"acc: {clf2.score(test_X2, test_y2)}")

In [ ]:
print(len(test_y1))
pred1_2 = clf1.predict(test_X2)
pred2_1 = clf2.predict(test_X1)


In [ ]:
# print(len(pred1_2))
# print(len(test_y2))


In [ ]:
j=0
for pred_1,testy1,pred_2,testy2,pred_12,pred_21 in zip(pred1,test_y1,pred2,test_y2,pred1_2,pred2_1):
    if pred_1 == testy1:
        j+=1
    elif pred_2 == testy2:
        j+=1
    elif pred_12 == testy2:
        j+=1
    elif pred_21 == testy1:
        j+=1

print(j)
score = j / 177
print(score)

In [ ]:
#グリッドサーチ込み
search_params = {
     'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
      'max_features'      : [3, 5, 10, 15, 20],
      'random_state'      : [2525],
      'n_jobs'            : [1],
      'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
      'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100]
}

gs1 = GridSearchCV(RandomForestClassifier(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=5,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs1 = gs1.fit(train_X1, train_y1)
#gs2 = gs1.fit(train_X2, train_y2)
pred_gs1 = gs1.predict(test_X1)
#pred_gs2 = gs2.predict(test_X2)
print(f"acc: {gs1.score(test_X1, test_y1)}")

In [ ]:
gs2 = gs1.fit(train_X2, train_y2)
pred_gs2 = gs2.predict(test_X2)
print(f"acc: {gs2.score(test_X2, test_y2)}")

In [ ]:
pred_gs1_2 = gs1.predict(test_X2)
pred_gs2_1 = gs2.predict(test_X1)

In [ ]:
js=0
for pred_1,testy1,pred_2,testy2,pred_12,pred_21 in zip(pred_gs1,test_y1,pred_gs2,test_y2,pred_gs1_2,pred_gs2_1):
    if pred_1 == testy1:
        js+=1
    elif pred_2 == testy2:
        js+=1
    elif pred_12 == testy2:
        js+=1
    elif pred_21 == testy1:
        js+=1

print(js)
score = js / 177
print(score)

In [ ]:
train_Xgen1 = train_data[:801]
train_ygen1 = Type1[:801]
train_ygen2 = Y_type2[:801]
#a_train, a_test = train_test_split(a, shuffle=False)
#(train_X2, test_X2 ,train_y2, test_y2) = train_test_split(, train_y2, test_size = 0.3, random_state = 666)
#print(train_X1)

test_Xgen1 = train_data[802:882]
test_ygen1 = Type1[802:882]
test_ygen2 = Y_type2[802:882]

In [ ]:
print(train_Xgen1)
print(train_ygen2)

In [ ]:
# search_params = {
#     'max_depth'         : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
#     'max_features'      : [3, 5, 10, 15, 20],
#      'n_estimators'      : [5, 10, 20, 30, 50, 100, 300],
#       'random_state'      : [2525],
#       'n_jobs'            : [1],
#       'min_samples_split' : [3, 5, 10, 15, 20, 25, 30, 40, 50, 100],
#     'min_impurity_decrease':[0.0]
# }
# gs1 = GridSearchCV(RandomForestClassifier(),
#                   search_params,
#                   cv=3,            # クロスバリデーションの分割数
#                   verbose=True,    # ログ表示
#                   n_jobs=-1)       # 並列処理

clf_gs1 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf_gs1 = clf_gs1.fit(train_Xgen1, train_ygen1)
pred_gs1 = clf_gs1.predict(test_Xgen1)
print(f"acc: {clf_gs1.score(test_Xgen1, test_ygen1)}")



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print(len(test_ygen1))
print(len(pred_gs1))
cm = confusion_matrix(test_ygen1, pred_gs1)
print(cm)
sns.heatmap(cm)

In [ ]:
clf_gs2 = RandomForestClassifier(
    bootstrap=True, #ランダムにデータを使う。falseだと順番ずつデータを取る。日付とかを重要視するらしい。
    class_weight=None, #不均衡を考慮しないならNone,考慮するならclass_weight='balanced'
    criterion='gini', #スプリットの品質を測定する機能。
    max_depth=15, #最大の深さ
    max_features=10, 
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    min_samples_leaf=1,
    min_samples_split=3,
    min_weight_fraction_leaf=0.0,
    n_estimators=300,
    n_jobs=1,
    oob_score=False,
    random_state=2525,
    verbose=0,
    warm_start=False)

clf_gs2 = clf_gs2.fit(train_Xgen1, train_ygen2)
pred_gs2 = clf_gs2.predict(test_Xgen1)
print(f"acc: {clf_gs2.score(test_Xgen1, test_ygen2)}")

In [ ]:
from pycm import ConfusionMatrix
print(len(test_ygen2))
print(len(pred_gs2))
cm = confusion_matrix(test_ygen1, pred_gs1)
print(cm)
sns.heatmap(cm)
# print(cm1)
# sns.heatmap(cm1)

mapping = {"Normal":0, "Fire":1, "Water":2, "Electric":3, "Grass":4, "Ice":5,
    "Fighting":6, "Poison":7, "Ground":8, "Flying":9, "Psychic":10, "Bug":11,
    "Rock":12, "Ghost":13, "Dragon":14, "Dark":15, "Steel":16,"Fairy":17 }


# gs2 = GridSearchCV(RandomForestClassifier(),
#                   search_params,
#                   cv=3,            # クロスバリデーションの分割数
#                   verbose=True,    # ログ表示
#                   n_jobs=-1) 
# gs2 = gs2.fit(train_Xgen1,train_ygen2)

In [ ]:
print(len(pred_gs2))
print(len(pred_gs1))
pred_gs1 = clf_gs1.predict(test_Xgen1)

In [ ]:
# print(gs1.score(test_Xgen1,test_ygen1))
# print(gs2.score(test_Xgen1,test_ygen2))

#print(f"acc: {gs2.score(train_Xgen2_test, train_ygen2_test2)}")
jj = 0
for gs1,test1,gs2,test2 in zip(pred_gs1 ,test_ygen1,pred_gs2,test_ygen2):
    if gs1 == test1:
        jj+=1
    elif gs2 == test2:
        jj+=1
    elif gs2 == test1:
        jj+=1
    elif gs1 == test2:
        jj+=1
print(jj)
score = jj / 80
print(score)

In [ ]:
gs_gs = GridSearchCV(RandomForestClassifier(),           # 対象の機械学習モデル
                  search_params,   # 探索パラメタ辞書
                  cv=5,            # クロスバリデーションの分割数
                  verbose=True,    # ログ表示
                  n_jobs=-1)       # 並列処理
gs_gs1 = gs_gs.fit(train_Xgen1, train_ygen1)
pred_gs_gs1 = gs_gs1.predict(test_Xgen1)
print(f"acc: {gs_gs1.score(test_Xgen1, test_ygen1)}")


In [ ]:
gs_gs2 = gs_gs.fit(train_Xgen1, train_ygen2)
pred_gs_gs2 = gs_gs2.predict(test_Xgen1)
print(f"acc: {gs_gs1.score(test_Xgen1, test_ygen2)}")

In [ ]:
jjs = 0
for gs1,test1,gs2,test2 in zip(pred_gs_gs1 ,test_ygen1,pred_gs_gs2,test_ygen2):
    if gs1 == test1:
        jjs+=1
    elif gs2 == test2:
        jjs+=1
    elif gs2 == test1:
        jjs+=1
    elif gs1 == test2:
        jjs+=1
print(jjs)
score = jjs / 80
print(score)

In [ ]:
#print(gs1.best_estimator_)
#print(gs2.best_estimator_)
print(gs_gs1.best_estimator_)
print(gs_gs2.best_estimator_)